In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep

In [45]:
sf = pd.read_csv('../data/tripadvisor_snowforecast_urls.csv')

In [46]:
url_list = sf['snow_forecast'].tolist()

In [47]:
def scrape_resort(url):
    
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    response = requests.get(url, headers=headers)
    search_page = response.text
    soup = BeautifulSoup(search_page,"html5lib")

    slop_stats = soup.find(class_='slope-picture')
    info_list = soup.find(class_='information-list')
    info_list = info_list.find_all('li')
    location = soup.find(class_='seo-line')
    info_list = soup.find_all(class_="information-list")[1]

    dic = {}
    
    dic['url'] = url

    # slope stats
    dic['top'] = slop_stats.find(class_='top-lift').text.replace('m','')
    dic['vertical_drop'] = slop_stats.find(class_='vertical-drop').text.replace('m','')
    dic['bottom'] = slop_stats.find(class_='bottom-lift').text.replace('m','')

    # info list
    dic['acres_of ski'] = info_list[0].text.replace('Acres of ski:','')
    dic['pistes'] = info_list[1].text.replace('Pistes:','')
    dic['snowmaking'] = info_list[2].text.replace('Snowmaking:','').replace("%",'')
    dic['halfpipes'] = info_list[3].text.replace('Halfpipes:','')
    dic['terrain_parks'] = info_list[4].text.replace('Terrain Parks:','')

    # location
    dic['country'] = location.find_all('a', href=True)[1].text.split(" - ")[0]
    if " - " in location.find_all('a', href=True)[1].text:
        dic['state_region'] = location.find_all('a', href=True)[1].text.split(" - ")[1]
    
    # resort website
    if "href" in str(info_list):
        resort_site = info_list.find('a', href=True).text
        dic['resort_website'] = info_list.find('a', href=True).text
    
    dic['resort_name'] = location.find(itemprop="name").text
    return(dic)

In [1]:
resorts = []
for url in url_list:
    res_info = scrape_resort(url)
    resorts.append(res_info)
    sleep(0.5)

In [49]:
resorts = pd.DataFrame.from_dict(resorts)

In [50]:
resorts = resorts[['url','resort_name','country','state_region','top','bottom','vertical_drop',
                   'acres_of ski','pistes','snowmaking','halfpipes','terrain_parks']]

In [51]:
resorts.head()

,resort_name,country,state_region,top,bottom,vertical_drop,acres_of ski,pistes,snowmaking,halfpipes,terrain_parks
0,Afriski Mountain Resort,Lesotho,NaN,3222,2917,305,10,2,-,-,1
1,Afton Alps,USA,Minnesota,107,0,107,-,-,-,-,-
2,Akakura Onsen,Japan,Niigata,1000,700,300,-,15,-,-,-
3,Alpbachtal,Austria,NaN,2025,813,1212,-,109,78,1,1
4,Alpe d'Huez,France,NaN,3350,1100,2250,583,116,10,-,2


In [52]:
# resorts.to_csv('snowforecast_resort_data.csv')